<a href="https://colab.research.google.com/github/EstebanmAcero/Neural_nets/blob/main/Ataques_adversarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Luis Esteban Molina Acero**
# **Manuel Vélez PhD.**


# Carga de librerías:
En esta sección, se importan las bibliotecas necesarias para realizar un ataque adversarial, incluyendo TensorFlow, NumPy, Pandas, Matplotlib, Keras, y otras

In [ ]:
# Importar las bibliotecas necesarias
import tensorflow         as tf  # Biblioteca principal de TensorFlow para aprendizaje automático y profundo
import numpy              as np  # Biblioteca para manipulación de arreglos y operaciones numéricas
import pandas             as pd  # Biblioteca para manipulación y análisis de datos tabulares
import matplotlib.pyplot  as plt # Biblioteca para crear gráficos
import keras             # API de alto nivel para construir y entrenar modelos de aprendizaje profundo
import requests          # Biblioteca para realizar solicitudes HTTP

# Importar módulos específicos de bibliotecas
from PIL                  import Image        # Módulo para trabajar con imágenes
from keras.preprocessing  import image        # Módulo para preprocesamiento de imágenes
from io                   import BytesIO      # Módulo para trabajar con transmisión de bytes
from keras                import backend as K # Módulo de backend de Keras para operaciones específicas

# Importar el modelo preentrenado InceptionV3 y funciones útiles
from keras.applications.inception_v3 import InceptionV3, decode_predictions, preprocess_input


A continuación, se carga el modelo preentrenado InceptionV3 y se muestra un resumen de su arquitectura para entender sus capas y parámetros.

In [ ]:
iv3 = InceptionV3()

In [ ]:
print(iv3.summary())

# Redimensionar o acomodar la imagen para el formato de predicción:
En esta parte, se descarga una imagen de una URL, se redimensiona a las dimensiones esperadas por el modelo (299x299 píxeles), y se prepara para la predicción.



In [ ]:
# Descargar y procesar la imagen desde una URL
url = "https://cdn.pixabay.com/photo/2021/09/20/23/03/car-6642036_1280.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Mostrar la imagen descargada
plt.imshow(img)
plt.axis('off')
plt.show()

# Redimensionar la imagen a la forma esperada por el modelo InceptionV3
img_resized = img.resize((299, 299))

# Convertir la imagen redimensionada a un formato adecuado para el modelo InceptionV3
x1 = image.img_to_array(img_resized)

# Normalizar la imagen
x1 /= 255
x1 -= 0.5
x1 *= 2

# Reshape la imagen para el formato de entrada del modelo
x1 = x1.reshape(1, x1.shape[0], x1.shape[1], x1.shape[2])

# Cargar el modelo InceptionV3
iv3 = InceptionV3()

# Realizar predicciones en la imagen preprocesada
y = iv3.predict(preprocess_input(x1), steps=1)




#Predicción en la imagen:
Aquí, se realiza una predicción en la imagen preprocesada utilizando el modelo InceptionV3 y se almacenan los resultados.

In [ ]:
# Suponiendo que 'y' es la salida de la red neuronal
predictions = decode_predictions(y)

# Check the number of columns in the data
n_columns = len(predictions[0])

# Crear un DataFrame con las predicciones
df_predictions = pd.DataFrame.from_records(predictions, columns=range(n_columns))

# Imprimir el DataFrame
(df_predictions)


In [ ]:


# Suponiendo que 'y' es la salida de la red neuronal
predictions = decode_predictions(y)

# Crear un DataFrame con las predicciones
df_predicciones = pd.DataFrame(columns=['Número', 'Etiqueta', 'Probabilidad'])

# Llenar el DataFrame con las predicciones decodificadas
for i, (imagenet_id, etiqueta, probabilidad) in enumerate(predictions[0]):
    df_predicciones = df_predicciones.append({
        'Número': i + 1,
        'Etiqueta': etiqueta,
        'Probabilidad': probabilidad
    }, ignore_index=True)

# Imprimir el DataFrame
print(df_predicciones)


# Romper el modelo (Ataque Adversario):
En esta sección, se lleva a cabo un ataque adversario para manipular la imagen de entrada y generar una imagen que engañe al modelo.

Vamos a romper el modelo y para eso se va a implementar la técnica de los ataques de adversarios.


Permite generar un tipo de datos diferentes de manera tal que confunde a la red en cuanto a la clasificación de las clases.

Lo que se va a realizar es que los parametros de entrada para optimizar o reducir el error van a cambiar, los pixeles van a  cambiar.




In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
# Obtener la capa de entrada y salida del modelo InceptionV3
inp_layer = iv3.layers[0].input
out_layer = iv3.layers[-1].output

# Especificar la clase objetivo para la adversarial attack
target_class = 951

# Definir la función de pérdida: la probabilidad de la clase objetivo
loss = out_layer[0, target_class]

# Calcular el gradiente de la pérdida con respecto a la entrada
grad = K.gradients(loss, inp_layer)[0]

# Crear una función para optimizar el gradiente y calcular la pérdida
optimize_gradient = K.function([inp_layer, K.learning_phase()], [grad, loss])

# Crear una copia de la imagen original para la adversarial attack
adv = np.copy(x1)

# Definir la perturbación máxima permitida
pert = 0.01
max_pert = x1 + 0.01
min_pert = x1 - 0.01

# Inicializar la variable de costo
cost = 0.0

# Iterar hasta que el costo sea mayor o igual a 0.95
while cost < 0.95:
    # Obtener el gradiente y la pérdida optimizados
    gr, cost = optimize_gradient([adv, 0])

    # Actualizar la imagen adversarial con el gradiente optimizado
    adv += gr

    # Limitar la imagen adversarial dentro de los límites de perturbación
    adv = np.clip(adv, min_pert, max_pert)
    adv = np.clip(adv, -1, 1)

    # Imprimir el costo en cada iteración
    print("Target cost:", cost)

# Crear una copia de la imagen adversarial después de la adversarial attack
hacked_img = np.copy(adv)


#Restablecer adv y mostrar la imagen nuevamente:
Finalmente, se realiza la inversión de las transformaciones realizadas durante el ataque adversario y se muestra la imagen adversarial generada.

In [ ]:
# Dividir todos los elementos de la matriz 'adv' por 2
adv /= 2

# Sumar 0.5 a todos los elementos de la matriz 'adv'
adv += 0.5

# Multiplicar todos los elementos de la matriz 'adv' por 255
adv *= 255


In [ ]:
# Mostrar la imagen adversarial después de las transformaciones
plt.imshow(adv[0].astype(np.uint8))

# Mostrar la imagen
plt.show()


In [ ]:
# Crear un objeto de imagen a partir de la matriz de píxeles de la imagen adversarial
im = Image.fromarray(adv[0].astype(np.uint8))

# Guardar la imagen en un archivo llamado "hacked.png"
im.save("./hacked.png")


# **Fallo en el intento de Optimización**

En el código, se ha buscado modularizar el proceso al crear funciones que permiten cargar imágenes tanto desde una ubicación específica como desde una URL. Estas imágenes se redimensionan y se utilizan para realizar predicciones utilizando el modelo InceptionV3, con los resultados almacenados en un DataFrame. Se implementa un ataque adversario para manipular la imagen y observar cómo afecta a las predicciones del modelo. Sin embargo, se ha observado que el modelo tiende a confundirse y no logra reconocer objetos en otras imágenes, a pesar de que se diseñó para adaptarse a cualquier imagen. Este comportamiento inesperado no se comprende y abordar las posibles razones detrás de esta confusión del modelo es tema de estudio.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from keras.preprocessing import image
from io import BytesIO
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3, decode_predictions, preprocess_input
import requests



def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.convert('RGB')
    img = img.resize((299, 299))  # Redimensionar la imagen
    img_array = image.img_to_array(img)
    return img_array



def preprocess_image(img):
    # Hacer una copia del array para evitar problemas de redimensionamiento
    img_copy = np.copy(img)

    # Redimensionar la imagen a la forma esperada por el modelo InceptionV3
    img_resized = Image.fromarray(img_copy.astype('uint8')).resize((299, 299))

    # Convertir la imagen redimensionada a un formato adecuado para el modelo InceptionV3
    x = image.img_to_array(img_resized)

    # Expandir las dimensiones para crear un lote (batch)
    x = np.expand_dims(x, axis=0)

    # Preprocesar la imagen utilizando la función preprocess_input de Keras
    x = preprocess_input(x)

    return x



def predict_image(model, img_array):
    # Redimensionar y preprocesar la imagen
    x = preprocess_input(np.expand_dims(img_array, axis=0))

    # Realizar predicciones en la imagen preprocesada
    predictions = model.predict(x)

    # Decodificar las predicciones
    decoded_predictions = decode_predictions(predictions)

    # Crear un DataFrame con las predicciones
    df_predictions = pd.DataFrame(columns=['Número', 'Etiqueta', 'Probabilidad'])

    # Llenar el DataFrame con las predicciones decodificadas
    for i, (imagenet_id, etiqueta, probabilidad) in enumerate(decoded_predictions[0]):
        df_predictions = df_predictions.append({
            'Número': i + 1,
            'Etiqueta': etiqueta,
            'Probabilidad': probabilidad
        }, ignore_index=True)

    return df_predictions





def adversarial_attack(model, img, target_class=951, perturbation=0.01):
    inp_layer = model.layers[0].input
    out_layer = model.layers[-1].output

    # Definir la función de pérdida: la probabilidad de la clase objetivo
    loss = out_layer[0, target_class]

    # Calcular el gradiente de la pérdida con respecto a la entrada
    grad = K.gradients(loss, inp_layer)[0]

    # Crear una función para optimizar el gradiente y calcular la pérdida
    optimize_gradient = K.function([inp_layer, K.learning_phase()], [grad, loss])

    # Crear una copia de la imagen original para la adversarial attack
    adv = np.copy(img)

    # Definir los límites de perturbación
    max_pert = img + perturbation
    min_pert = img - perturbation

    # Inicializar la variable de costo
    cost = 0.0

    # Iterar hasta que el costo sea mayor o igual a 0.95
    while cost < 0.95:
        gr, cost = optimize_gradient([adv, 0])
        adv += gr
        adv = np.clip(adv, min_pert, max_pert)
        adv = np.clip(adv, -1, 1)
        print("Target cost:", cost)

    # Deshacer las transformaciones en la imagen adversarial
    adv /= 2
    adv += 0.5
    adv *= 255

    return adv


# Cargar el modelo InceptionV3
model_iv3 = InceptionV3()

# URL de la imagen a procesar
url = "https://images.unsplash.com/photo-1583121274602-3e2820c69888?q=80&w=3570&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Cargar la imagen desde la URL
image_from_url = load_image_from_url(url)

# Predicciones en la imagen original
df_predictions_original = predict_image(model_iv3, image_from_url)
print("Predicciones en la imagen original:")
print(df_predictions_original)

# Mostrar la imagen original
plt.imshow(image_from_url)
plt.axis('off')
plt.title("Imagen Original")
plt.show()

# Predicciones en la imagen original
df_predictions_original = predict_image(model_iv3, image_from_url)
print("Predicciones en la imagen original:")
print(df_predictions_original)

# Procesar la imagen para el ataque adversario
processed_image = preprocess_image(image_from_url)
tf.compat.v1.disable_eager_execution()
# Ataque adversario
adversarial_image = adversarial_attack(model_iv3, processed_image)

# Mostrar la imagen adversarial
plt.imshow(adversarial_image[0].astype(np.uint8))
plt.axis('off')
plt.title("Imagen Adversarial")
plt.show()

# Predicciones en la imagen adversarial
df_predictions_adversarial = predict_image(model_iv3, adversarial_image)
print("Predicciones en la imagen adversarial:")
print(df_predictions_adversarial)


In [ ]:

def predecir_imagen(ruta_imagen):
    # Cargar la imagen desde la ruta
    img = image.load_img(ruta_imagen, target_size=(299, 299))

    # Convertir la imagen a un arreglo NumPy usando img_to_array
    x1 = image.img_to_array(img)

    # Realizar el preprocesamiento de la imagen
    x1 /= 255
    x1 -= 0.5
    x1 *= 2
    x1 = np.expand_dims(x1, axis=0)  # Añadir una dimensión adicional para representar el lote (batch)

    # Crear un modelo InceptionV3 preentrenado
    iv3 = InceptionV3()

    # Realizar la predicción
    y = iv3.predict(preprocess_input(x1))

    # Decodificar las predicciones
    predicciones_decodificadas = decode_predictions(y)

    # Imprimir las predicciones decodificadas
    for i, (imagenet_id, etiqueta, probabilidad) in enumerate(predicciones_decodificadas[0]):
        print(f"{i + 1}: {etiqueta} ({probabilidad:.2f})")

# Ejemplo de uso
ruta_imagen_ejemplo = '/content/hacked.png'
predecir_imagen(ruta_imagen_ejemplo)


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

def process_image_url(url):
    # Download the image using requests
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    # Resize the image to the target size
    img = img.resize((299, 299))

    # Convert the PIL image to a NumPy array using image.img_to_array
    x1 = img_to_array(img)

    # Normalize the image
    x1 /= 255
    x1 -= 0.5
    x1 *= 2

    # Reshape the array for the model input
    x1 = x1.reshape(1, x1.shape[0], x1.shape[1], x1.shape[2])

    # Load the InceptionV3 model
    iv3 = InceptionV3()

    # Make predictions
#    y = iv3.predict(preprocess_input(x1))
    y = iv3.predict(preprocess_input(x1), steps=1)


    # Decode and print predictions
    predictions = decode_predictions(y)
    #print(predictions)
    # Crear un DataFrame con las predicciones
    df_predicciones = pd.DataFrame(columns=['Número', 'Etiqueta', 'Probabilidad'])

    # Llenar el DataFrame con las predicciones decodificadas
    for i, (imagenet_id, etiqueta, probabilidad) in enumerate(predictions[0]):

        df_predicciones = pd.concat([df_predicciones, pd.DataFrame({
            'Número': [i + 1],
            'Etiqueta': [etiqueta],
            'Probabilidad': [probabilidad]
        })], ignore_index=True)

    return df_predicciones



url = "https://images.unsplash.com/photo-1583121274602-3e2820c69888?q=80&w=3570&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Mostrar la imagen antes de procesarla
response = requests.get(url)
img = Image.open(BytesIO(response.content))
plt.imshow(img)
plt.axis('off')
plt.show()

a = process_image_url(url)
a
